# Natural Language Processing
## Assignment 2: Training NMT model with fairseq

### 1. Introduction

- 본 과제의 목적은 대표적인 pytorch library 중 하나인 fairseq을 이용해 번역 모델을 학습하는 방법을 배우는 것입니다.
- 일반적으로 우리는 해당 task와 관련되어 널리 알려진 library를 이용해 모델을 구현하고 tuning하게 됩니다. 자연어 처리 관련 여러 라이브러리가 있지만 번역 task에서 가장 자주 활용되고 있는 fairseq을 소개해드리고자 합니다. fairseq은 pytorch를 개발하고 있는 facebook에서 작업 중인 오픈소스 프로젝트 입니다. library의 이름처럼 sequence를 다루는 데에 필요한 여러 모델과 전처리, 평가 관련 코드를 포함해 인기가 많은 library 중 하나입니다.
- 이번 시간에는 해당 library의 [docs](https://fairseq.readthedocs.io/en/latest/)를 직접 읽어보면서 목표 perplexity/BLEU를 달성하는 것이 목표입니다. 프로젝트에 대한 대략적인 설명과 관련 docs 링크를 함께 제공해드리겠습니다. 주어진 데이터에 대해 **BLEU score 25 이상**을 달성해보세요 !
- ***먼저 colab 상단 탭 runtime -> change runtime type에서 hardware accelerator를 GPU로 변경해주세요***

In [1]:
# 필요 패키지 설치 (참고: 느낌표를 앞에 붙이면 해당 코드가 terminal에서 실행됩니다.)
!pip install fastBPE sacremoses subword_nmt hydra-core omegaconf fairseq

     |████████████████████████████████| 890kB 23.3MB/s 
     |████████████████████████████████| 133kB 55.3MB/s 
     |████████████████████████████████| 1.7MB 51.4MB/s 
     |████████████████████████████████| 112kB 34.9MB/s 
     |████████████████████████████████| 645kB 44.5MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp36-cp36m-linux_x86_64.whl size=481522 sha256=ae953cf3a5248906d0149d8778b71f51a1a27e84f715a31e4d45da2c0e05c11f
  Stored in directory: /root/.cache/pip/wheels/f3/0c/9c/fc62058b4d473a5602bcd3d3edfece796f123875379ea82d79
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=072a2a5d3a769ee086a7f76b41875dd49cd00647c2848e188fc27ef53a6700c6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-cp36-none-any.whl size=141231 sha256=19ce84439

In [2]:
# clone fairseq git
!git clone https://github.com/pytorch/fairseq.git
# iwslt14 데이터 준비 (https://github.com/pytorch/fairseq/blob/master/examples/translation/prepare-iwslt14.sh)
!bash fairseq/examples/translation/prepare-iwslt14.sh

Cloning into 'fairseq'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 21291 (delta 32), reused 44 (delta 19), pack-reused 21216
Receiving objects: 100% (21291/21291), 9.52 MiB | 27.15 MiB/s, done.
Resolving deltas: 100% (15895/15895), done.
Cloning Moses github repository (for tokenization scripts)...
Cloning into 'mosesdecoder'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 147592 (delta 5), reused 11 (delta 3), pack-reused 147572
Receiving objects: 100% (147592/147592), 129.76 MiB | 22.55 MiB/s, done.
Resolving deltas: 100% (114031/114031), done.
Cloning Subword NMT repository (for BPE pre-processing)...
Cloning into 'subword-nmt'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 580 (

## 2. Library reference
1. [tasks](https://fairseq.readthedocs.io/en/latest/tasks.html)
 - translation task와 language modeling task가 있고 나머지 sequence를 다루는 task는 register_task() function decorator를 이용해 등록할 수 있습니다.
2. [models](https://fairseq.readthedocs.io/en/latest/models.html)
 - 모델은 CNN, LSTM, Transformer 기반 모델들이 분류가 되어 있습니다. transformer 모델쪽 코드가 꼼꼼히 잘되어 있습니다. 새로운 모델을 등록하기 위해서는 register_model() function decorator를 이용할 수 있습니다.
3. [criterions](https://fairseq.readthedocs.io/en/latest/criterions.html)
 - 모델 학습을 위한 다양한 loss들이 구현되어 있습니다.
4. [optimizers](https://fairseq.readthedocs.io/en/latest/optim.html)
 - 모델 학습을 위한 다양한 optimizer들이 구현되어 있습니다.
5. l[earning rate schedulers](https://fairseq.readthedocs.io/en/latest/lr_scheduler.html)
 - 모델의 더 나은 학습을 위한 다양한 learning rate scheduler들이 구현되어 있습니다.
6. [data loading and utilities](https://fairseq.readthedocs.io/en/latest/data.html)
 - 전처리 및 데이터 관련 다양한 class들이 구현되어 있습니다.
7. [modules](https://fairseq.readthedocs.io/en/latest/modules.html)
 - 앞의 6군데에 속하지 못한(?) 다양한 모듈들이 구현되어 있습니다.

## 3. [Command-line Tools](https://fairseq.readthedocs.io/en/latest/command_line_tools.html)
fairseq은 학습과 평가를 쉽게할 수 있는 command-line tool을 제공하고 있습니다
각각의 커맨드라인에 대한 설명은 위 링크에 자세히 나와있으니 참고해주시기 바랍니다.
1. fairseq-preprocess
 - 데이터 학습을 위한 vocab을 만들고 data를 구성합니다.
2. fairseq-train
 - 여러 gpu 또는 단일 gpu에서 모델을 학습시킵니다.
3. fairseq-generate
 - 학습된 모델을 이용해 전처리된 데이터를 번역합니다.
4. fairseq-interactive
 - 학습된 모델을 이용해 raw 데이터를 번역합니다.
5. fairseq-score
 - 학습된 모델이 생성한 문장과 정답 문장을 비교해 bleu score를 산출합니다.
6. fairseq-eval-lm
 - language model을 평가할 수 있는 command입니다.


In [3]:
# 예시 코드를 이용해 직접 전처리부터 평가까지 진행해보겠습니다.
# source-lang: source language
# target-lang: target language
# trainpref: train file prefix
# validpref: valid file prefix
# testpref: test file prefix
# destdir: destination dir
!fairseq-preprocess --source-lang de --target-lang en --trainpref ./iwslt14.tokenized.de-en/train --validpref ./iwslt14.tokenized.de-en/valid --testpref ./iwslt14.tokenized.de-en/test --destdir ./iwslt14.tokenized.de-en/

2021-02-17 18:22:15 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='./iwslt14.tokenized.de-en/', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='de', srcdict=None, target_lang='en', task='translation', tensorboard_logdir=None, testpref='./iwslt14.tokenized.de-en/test', tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, tok

In [4]:
# 모델 학습
# (참고: 모델을 동시에 여러개 학습 시키고 싶으신 분들은 노트북 파일을 드라이브에 여러개 복사해서 각 파일마다 실행하면 여러 모델을 동시에 학습시킬 수 있습니다.)
# --arch: architecture
# --optimizer: optimizer {adadelta, adam, adafactor, adagrad, lamb, composite, nag, adamax, sgd}
# --clip-norm: clip threshold of gradients
# --lr: learning rate
# --lr-scheduler: learning rate scheduler {pass_through, cosine, reduce_lr_on_plateau, fixed, triangular, polynomial_decay, tri_stage, manual, inverse_sqrt}
# --criterion loss function {sentence_prediction, ctc, adaptive_loss, label_smoothed_cross_entropy, composite_loss, nat_loss, masked_lm, sentence_ranking, legacy_masked_lm_loss, cross_entropy, model, wav2vec, label_smoothed_cross_entropy_with_alignment, vocab_parallel_cross_entropy}
# --max-tokens: maximum number of tokens in a batch
# --max-epoch: maximum number of training epoch

!fairseq-train ./iwslt14.tokenized.de-en/ --arch transformer_iwslt_de_en --optimizer adam --clip-norm 0.1 --dropout 0.1 --lr 5e-4 --lr-scheduler inverse_sqrt --criterion label_smoothed_cross_entropy_with_alignment --max-tokens 4096 --max-epoch 5 --reset-optimizer --reset-lr-scheduler

2021-02-17 18:23:36 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.999)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, alignment_lambda=0.05, all_gather_list_size=16384, arch='transformer_iwslt_de_en', attention_dropout=0.0, batch_size=None, batch_size_valid=None, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.1, cpu=False, criterion='label_smoothed_cross_entropy_with_alignment', cross_self_attention=False, curriculum=0, data='./iwslt14.tokenized.de-en/', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=4, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=1024, decoder_input_dim=512, decoder_layerdrop=0, decoder_layers=6, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=False, decoder_o

In [5]:
 # 예측 문장 생성 및 평가
 # checkpoints 폴더에 epoch마다 모델이 저장되고 best checkpoint는 checkpoint_best.pt라는 이름으로 저장됩니다
!fairseq-generate ./iwslt14.tokenized.de-en --path ./checkpoints/checkpoint_best.pt --beam 5 --remove-bpe

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
D-5565	-0.7678383588790894	and because the government ultimately pushed out their power from us -- remember , reminiscent of us , &quot; we humans ? &quot; -- how we think about it is going to affect how this change is going to happen .
P-5565	-0.0523 -0.7069 -0.7622 -0.1404 -1.0379 -4.3679 -0.3764 -1.5889 -1.7715 -1.0204 -0.4318 -0.1297 -0.4139 -1.2987 -0.6630 -2.3465 -1.6028 -0.0038 -0.2318 -0.3414 -0.6207 -0.2546 -0.6211 -1.4390 -0.8051 -0.0828 -0.4748 -0.2318 -0.2494 -0.0759 -0.2310 -0.8946 -2.2940 -0.4856 -0.0113 -0.7731 -1.1981 -1.0325 -0.1713 -0.9985 -0.4357 -0.2891 -1.4309 -0.1615 -0.0021
S-4769	sind sie verlässlich , stabil , familiär , sicher , geborgen , heilig , bedacht und weise wie der dalai lama oder yoda ?
T-4769	are you reliable , stable , familiar , safe , secure , sacred , contemplative or wise like the dalai lama or yoda ?
H-4769	-0.7170787453651428	are they reliable , robust , family , secure , robust , robust , holy , arguably l

In [6]:
# 여러분만의 모델을 학습해 목표 BLEU score를 달성해주세요 ! 
# command line tools: https://fairseq.readthedocs.io/en/latest/command_line_tools.html
# github: https://github.com/pytorch/fairseq